In [70]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
data = pd.read_csv('Segmentation Data/transactions.csv', sep=';')
codes = pd.read_csv('Segmentation Data/codes.csv', sep=';')
types = pd.read_csv('Segmentation Data/types.csv',sep=';')
train = pd.read_csv('Segmentation Data/train_set.csv', sep=';')
test = pd.read_csv('Segmentation Data/test_set.csv', sep=';')

In [4]:
data_merged = data.merge(train)
data_merged.drop_duplicates(inplace = True)
data_merged[['day', 'datetime']] = data_merged['datetime'].str.split(' ', 1, expand=True)
gal = data_merged.copy()

In [6]:
# def mean_time2(nate):
#     day = nate[:-8]
#     #time = str(nate[-8:])
#     #date = (int(time.split(':')[0]), int(time.split(':')[1]), int(time.split(':')[2]))
#     #return round(int(day) * 24 * 3600 + date[0] * 3600 + date[1] * 60 + date[2], 1)
#     return int(day)
# gal['datetime'] = gal['datetime'].apply(mean_time2)
gal

,client_id,datetime,code,type,sum,target,day
0,96372458,06:33:15,6011,2010,-561478.94,0,421
1,96372458,06:16:18,6011,7010,224591.58,0,270
2,96372458,10:48:35,4829,2370,-11229.58,0,97
3,96372458,07:33:38,4829,2330,-417695.42,0,445
4,96372458,09:33:46,4814,1030,-2245.92,0,4
...,...,...,...,...,...,...,...
91821,82133712,13:23:10,6011,2010,-17967.33,1,424
91822,82133712,09:38:10,6536,6110,179673.26,1,232
91823,12289409,11:13:33,6011,2010,-132509.03,0,108
91824,71829751,14:50:37,6011,2010,-44918.32,0,72


In [37]:
data_merged2 = data_merged.copy()
data_merged2['monetary'] = data_merged2['sum'].apply(lambda x: float(abs(x)))
groupedGal = data_merged2.groupby('client_id').sum().drop(['code', 'type', 'target', 'sum'], axis = 1)
groupedGal['recency'] = data_merged2.groupby('client_id').max()['day'].apply(lambda x: 457 - int(x)).to_list()
groupedGal['frequency'] = data_merged2.groupby('client_id').count()['day'].to_list()
groupedGal

,monetary,recency,frequency
client_id,,,
22899,119842.08,391,9
27914,83098.89,165,4
28753,3937349.75,366,13
38084,2810132.96,377,26
49793,1436315.95,376,21
...,...,...,...
99900908,72052.13,146,5
99911226,667589.93,363,12
99967537,336887.37,125,1


In [52]:
groupedGal["frequencyScore"] = pd.cut(groupedGal["frequency"], 
                                       bins=[-1, 
                                         np.percentile(groupedGal["frequency"], 25), 
                                             np.percentile(groupedGal["frequency"], 50), 
                                             np.percentile(groupedGal["frequency"], 75), 
                                             groupedGal["frequency"].max()], 
                                       labels=[1, 2, 3, 4]).astype("int")
groupedGal["recencyScore"] = pd.cut(groupedGal["recency"], 
                                       bins=[-1, 
                                         np.percentile(groupedGal["recency"], 25), 
                                             np.percentile(groupedGal["recency"], 50), 
                                             np.percentile(groupedGal["recency"], 75), 
                                             groupedGal["recency"].max()], 
                                       labels=[4, 3, 2, 1]).astype("int")
groupedGal["monetaryScore"] = pd.cut(groupedGal["monetary"], 
                                       bins=[-1, 
                                         np.percentile(groupedGal["monetary"], 25), 
                                             np.percentile(groupedGal["monetary"], 50), 
                                             np.percentile(groupedGal["monetary"], 75), 
                                             groupedGal["monetary"].max()], 
                                       labels=[1, 2, 3, 4]).astype("int")
groupedGal['rfm'] = groupedGal["monetaryScore"] + groupedGal["recencyScore"] + groupedGal["frequencyScore"]
groupedGal

,monetary,recency,frequency,frequencyScore,recencyScore,monetaryScore,rfm
client_id,,,,,,,
22899,119842.08,391,9,2,1,1,4
27914,83098.89,165,4,1,3,1,5
28753,3937349.75,366,13,3,2,4,9
38084,2810132.96,377,26,4,2,4,10
49793,1436315.95,376,21,4,2,4,10
...,...,...,...,...,...,...,...
99900908,72052.13,146,5,1,3,1,5
99911226,667589.93,363,12,3,3,3,9
99967537,336887.37,125,1,1,3,2,6


In [61]:
fig = px.scatter_3d(groupedGal, x='frequencyScore', y='recencyScore', z='monetaryScore',
                   color = 'rfm')
fig.show()

In [81]:
data

,client_id,datetime,code,type,sum
0,96372458,421 06:33:15,6011,2010,-561478.94
1,24567813,377 17:20:40,6011,7010,67377.47
2,21717441,55 13:38:47,6011,2010,-44918.32
3,14331004,263 12:57:08,6011,2010,-3368873.66
4,85302434,151 10:34:12,4814,1030,-3368.87
...,...,...,...,...,...
130034,15836839,147 11:50:53,5411,1010,-26344.59
130035,28369355,305 11:59:34,4829,2330,-24705.07
130036,40949707,398 21:13:58,5411,1110,-40353.72
130037,7174462,409 13:58:14,5411,1010,-25536.06


In [68]:
rfmData = groupedGal[['rfm', 'frequencyScore', 'recencyScore','monetaryScore']]
rfmData

,rfm,frequencyScore,recencyScore,monetaryScore
client_id,,,,
22899,4,2,1,1
27914,5,1,3,1
28753,9,3,2,4
38084,10,4,2,4
49793,10,4,2,4
...,...,...,...,...
99900908,5,1,3,1
99911226,9,3,3,3
99967537,6,1,3,2


In [80]:
groupedGal.groupby('rfm').mean()

,monetary,recency,frequency,frequencyScore,recencyScore,monetaryScore
rfm,,,,,,
3,5.675118e+04,414.644578,3.060241,1.000000,1.000000,1.000000
4,1.095635e+05,396.951327,5.345133,1.438053,1.331858,1.230088
5,1.454744e+05,294.283019,5.104499,1.445573,2.137881,1.416546
6,2.337350e+05,237.460058,6.713186,1.664100,2.596728,1.739172
7,4.684185e+05,251.333333,10.279960,2.212487,2.514602,2.272910
8,8.795773e+05,258.975108,13.847403,2.638528,2.568182,2.793290
9,1.569193e+06,278.886986,20.166667,3.192922,2.557078,3.250000
10,2.547197e+06,275.650986,25.484067,3.550835,2.796662,3.652504
11,6.381893e+06,258.978892,51.517150,3.788918,3.292876,3.918206
